In [7]:
# conversation buffer memory
"""
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context(
	{"input": "Hi!"},
	{"output": "How are you?"}
)
memory.load_memory_variables({})
"""


In [ ]:
# conversation buffer window memory : FIFO 형태, 데이터가 크기 이상으로 들어오면 가장 먼저 들어온 항목이 삭제된다.
"""
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
	return_messages=True,
	k=4
)

def add_message(input, output):
	memory.save_context({"input": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
memory.load_memory_variables({})
add_message(5, 5)
memory.load_memory_variables({})
"""

In [ ]:
# conversation summary memory
"""
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
	memory.save_context({"input": input}, {"output": output})

def get_history():
	return memory.load_memory_variables({})

add_message("Hi, I'm Lee, I live in South Korea", "Wow, that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!")
get_history()
"""

In [ ]:
# conversation summary buffer memory
"""
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=150,
	return_messages=True
)

def add_message(input, output):
	memory.save_context({"input": input}, {"output": output})

def get_history():
	return memory.load_memory_variables({})

add_message("Hi, I'm Lee, I live in South Korea", "Wow, that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!")
get_history()
"""

In [32]:
# conversation KG memory : 대화 내용을 분석.

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(
	llm=llm,
	return_messages=True
)

def add_message(input, output):
	memory.save_context({"input": input}, {"output": output})

add_message("Hi, I'm Lee, I live in South Korea", "Wow, that is so cool!")
memory.load_memory_variables({"input": "who is Lee"})

add_message("Lee likes pizza", "pizza is nice")
memory.load_memory_variables({"input": "what does Lee like"})


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "history": "Human: Hi, I'm Lee, I live in South Korea\nAI: Wow, that is so cool!",
  "input": "Hi, I'm Lee, I live in South Korea",
  "verbose": true
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the last line of conversation. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.\n\nEXAMPLE\nConversation history:\nPerson #1: Did you hear aliens landed in Area 51?\nAI: No, I didn't hear that. What do

{'history': [SystemMessage(content='On Lee: Lee lives in South Korea. Lee likes pizza.')]}

In [22]:
# memory on llm chain

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
	temperature=0.1,
)
memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=120,
	memory_key="chat_history",
	return_messages=True,
)

template = """ 

	You are a helpful AI talking to a human.

	{chat_history}
	Human: {question}
	You:
"""

chatPrompt = ChatPromptTemplate.from_messages([
	("system", "You are a helpful AI talking to a human."),
	MessagesPlaceholder(variable_name="chat_history", ),
	("human", "{question}"),
])

chain = LLMChain(
	llm=llm,
	memory=memory,
	#1. prompt=PromptTemplate.from_template("{question}"),
	#2. prompt=PromptTemplate.from_template(template),
	prompt=chatPrompt,
	verbose=True,
)

chain.predict(question="My name is Lee")
memory.load_memory_variables({})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "My name is Lee",
  "chat_history": []
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI talking to a human.\nHuman: My name is Lee"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello Lee! How can I assist you today?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Hello Lee! How can I assist you today?",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_to

{'chat_history': [HumanMessage(content='My name is Lee'),
  AIMessage(content='Hello Lee! How can I assist you today?')]}

In [23]:
chain.predict(question="I live in Seongnam-si")
memory.load_memory_variables({})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI talking to a human.\nHuman: My name is Lee\nAI: Hello Lee! How can I assist you today?\nHuman: I live in Seongnam-si"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "That's great to know, Lee! Is there anything specific you would like to know or talk about regarding Seongnam-si?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "That's great to know, Lee! Is there anything specific you would like to 

{'chat_history': [HumanMessage(content='My name is Lee'),
  AIMessage(content='Hello Lee! How can I assist you today?'),
  HumanMessage(content='I live in Seongnam-si'),
  AIMessage(content="That's great to know, Lee! Is there anything specific you would like to know or talk about regarding Seongnam-si?")]}

In [24]:
chain.predict(question="What is my name?")
memory.load_memory_variables({})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI talking to a human.\nHuman: My name is Lee\nAI: Hello Lee! How can I assist you today?\nHuman: I live in Seongnam-si\nAI: That's great to know, Lee! Is there anything specific you would like to know or talk about regarding Seongnam-si?\nHuman: What is my name?"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Your name is Lee. How can I assist you further, Lee?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "

{'chat_history': [HumanMessage(content='My name is Lee'),
  AIMessage(content='Hello Lee! How can I assist you today?'),
  HumanMessage(content='I live in Seongnam-si'),
  AIMessage(content="That's great to know, Lee! Is there anything specific you would like to know or talk about regarding Seongnam-si?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Lee. How can I assist you further, Lee?')]}

In [29]:
# LCEL based memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
	temperature=0.1,
)
memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=120,
	return_messages=True,
)

template = """ 

	You are a helpful AI talking to a human.

	{history}
	Human: {question}
	You:
"""

prompt = ChatPromptTemplate.from_messages([
	("system", "You are a helpful AI talking to a human."),
	MessagesPlaceholder(variable_name="history", ),
	("human", "{question}"),
])

def load_memory(_):
	return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
	result = chain.invoke({"question": question})
	memory.save_context(
		{"input": question}, 
		{"output": result.content}
	)
	print(result)

invoke_chain("My name is Lee")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "My name is Lee"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "My name is Lee"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] Entering Chain run with input:
{
  "question": "My name is Lee"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "history": []
}
[chain/start] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "My name is Lee",
  "history": []
}
[chain/end] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
   

In [30]:
invoke_chain("I live in seongnam-si")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "I live in seongnam-si"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "I live in seongnam-si"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] Entering Chain run with input:
{
  "question": "I live in seongnam-si"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] s] Exiting Chain run with output:
{
  "output": [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "HumanMessage"
      ],
      "kwargs": {
        "content": "My name is Lee"
      }
    },
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "AIMessage"
      ],
      "kwargs": {
        "content": "Hello Lee! How can I assist yo

In [31]:
invoke_chain("what is my name?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] s] Exiting Chain run with output:
{
  "output": [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "HumanMessage"
      ],
      "kwargs": {
        "content": "My name is Lee"
      }
    },
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "AIMessage"
      ],
      "kwargs": {
        "content": "Hello Lee! How can I assist you today?"
     